In [27]:
import os
import pandas as pd

REMOVE_COLUMNS_REGION = ["CODIGO (WMO):", "DATA DE FUNDACAO:", "ESTACAO:"]
GET_COLUMNS_CLIMATE = [
    "Data",
    "Hora UTC",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",
    "VENTO, VELOCIDADE HORARIA (m/s)",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",
    "RADIACAO GLOBAL (Kj/m²)",
    "UMIDADE RELATIVA DO AR, HORARIA (%)",
]


In [28]:
climate_dir = "./data/2022"
list_df_station = []
list_df_climate = []

for file_name in os.listdir(climate_dir):
    df_station = pd.read_csv(
        f"{climate_dir}/{file_name}",
        delimiter=";",
        encoding="Windows 1252",
        nrows=7,
    ).set_index("REGIAO:").transpose().reset_index(drop=True)
    df_station = df_station.drop(columns=REMOVE_COLUMNS_REGION)

    df_climate = pd.read_csv(
        f"{climate_dir}/{file_name}",
        delimiter=";",
        encoding="Windows 1252",
        skiprows=8,
    ).dropna(how="all", subset=GET_COLUMNS_CLIMATE[2:]).get(GET_COLUMNS_CLIMATE)

    list_df_station.append(df_station)
    list_df_climate.append(df_climate)

df_all_station = pd.concat(list_df_station, ignore_index=True)
df_all_station.to_csv("./data/inmet_regiao.csv", sep=";")

df_all_climate = pd.concat(list_df_climate, ignore_index=True)
df_all_climate.to_csv("./data/inmet_dados_meteorologicos.csv", sep=";")
